# RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17464789/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 24s 29ms/step - loss: 0.6891 - accuracy: 0.5122 - val_loss: 0.6784 - val_accuracy: 0.5408
Epoch 2/10
782/782 [==============================] - 13s 17ms/step - loss: 0.6536 - accuracy: 0.5994 - val_loss: 0.6383 - val_accuracy: 0.6966
Epoch 3/10
782/782 [==============================] - 12s 16ms/step - loss: 0.6137 - accuracy: 0.7028 - val_loss: 0.6152 - val_accuracy: 0.6998
Epoch 4/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5893 - accuracy: 0.7414 - val_loss: 0.5992 - val_accuracy: 0.7304
Epoch 5/10
782/782 [==============================] - 12s 16ms/step - loss: 0.5706 - accuracy: 0.7620 - val_loss: 0.5906 - val_accuracy: 0.7333
Epoch 6/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5553 - accuracy: 0.7717 - val_loss: 0.5811 - val_accuracy: 0.7423
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5415 - accuracy: 0.7832 - val_loss: 0.5756 - val_accuracy:

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.4841 - accuracy: 0.7678
Test score: 0.48414671421051025
Test accuracy: 0.767799973487854


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [11]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [12]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 25s 31ms/step - loss: 0.6786 - accuracy: 0.5318 - val_loss: 0.6356 - val_accuracy: 0.6038
Epoch 2/10
782/782 [==============================] - 24s 31ms/step - loss: 0.5967 - accuracy: 0.7222 - val_loss: 0.5843 - val_accuracy: 0.7408
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 0.5507 - accuracy: 0.7864 - val_loss: 0.5641 - val_accuracy: 0.8010
Epoch 4/10
782/782 [==============================] - 25s 32ms/step - loss: 0.5223 - accuracy: 0.8132 - val_loss: 0.5414 - val_accuracy: 0.7902
Epoch 5/10
782/782 [==============================] - 24s 31ms/step - loss: 0.4983 - accuracy: 0.8278 - val_loss: 0.5317 - val_accuracy: 0.7899
Epoch 6/10
782/782 [==============================] - 24s 30ms/step - loss: 0.4784 - accuracy: 0.8402 - val_loss: 0.5076 - val_accuracy: 0.8107
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.4617 - accuracy: 0.8469 - val_loss: 0.5223 - val_accuracy:

In [15]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 22s 26ms/step - loss: 0.6794 - accuracy: 0.5422 - val_loss: 0.6647 - val_accuracy: 0.7052
Epoch 2/10
782/782 [==============================] - 19s 24ms/step - loss: 0.6129 - accuracy: 0.7143 - val_loss: 0.5926 - val_accuracy: 0.7589
Epoch 3/10
782/782 [==============================] - 20s 26ms/step - loss: 0.5679 - accuracy: 0.7739 - val_loss: 0.5673 - val_accuracy: 0.7880
Epoch 4/10
782/782 [==============================] - 27s 35ms/step - loss: 0.5409 - accuracy: 0.7997 - val_loss: 0.5497 - val_accuracy: 0.7953
Epoch 5/10
782/782 [==============================] - 21s 27ms/step - loss: 0.5162 - accuracy: 0.8134 - val_loss: 0.5355 - val_accuracy: 0.7666
Epoch 6/10
782/782 [==============================] - 19s 25ms/step - loss: 0.4240 - accuracy: 0.8103 - val_loss: 0.4443 - val_accuracy: 0.7930
Epoch 7/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3962 - accuracy: 0.8245 - val_loss: 0.4229 - val_accuracy:

In [19]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3300 - accuracy: 0.8586 - val_loss: 0.4475 - val_accuracy: 0.7960
Epoch 2/10
782/782 [==============================] - 19s 24ms/step - loss: 0.3215 - accuracy: 0.8623 - val_loss: 0.3732 - val_accuracy: 0.8315
Epoch 3/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3123 - accuracy: 0.8686 - val_loss: 0.3677 - val_accuracy: 0.8349
Epoch 4/10
782/782 [==============================] - 19s 24ms/step - loss: 0.3062 - accuracy: 0.8707 - val_loss: 0.3650 - val_accuracy: 0.8345
Epoch 5/10
782/782 [==============================] - 21s 26ms/step - loss: 0.2994 - accuracy: 0.8748 - val_loss: 0.3640 - val_accuracy: 0.8354
Epoch 6/10
782/782 [==============================] - 19s 24ms/step - loss: 0.2945 - accuracy: 0.8768 - val_loss: 0.3810 - val_accuracy: 0.8304
Epoch 7/10
782/782 [==============================] - 21s 26ms/step - loss: 0.2898 - accuracy: 0.8805 - val_loss: 0.3574 - val_accuracy: